In [9]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.utilities import ArxivAPIWrapper


In [10]:
# Use the inbuild tools
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200))
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200))

In [9]:
arxiv.name

'arxiv'

In [10]:
wikipedia.name

'wikipedia'

In [7]:
tools = [wikipedia, arxiv]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\machine-learning\\huggingface-langchain\\langchain-with-krish\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000))]

In [1]:
# create custom tools
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
loader = WebBaseLoader(web_path="https://www.geeksforgeeks.org/machine-learning/ml-machine-learning/")
doc = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
documents = text_splitter.split_documents(documents=doc)
documents

[Document(metadata={'source': 'https://www.geeksforgeeks.org/machine-learning/ml-machine-learning/', 'title': 'What is Machine Learning? - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content='What is Machine Learning? - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content'),
 Document(metadata={'source': 'https://www.geeksforgeeks.org/machine-learning/ml-machine-learning/', 'title': 'What is Machine Learning? - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, compe

In [4]:
embedding = HuggingFaceEmbeddings(model='all-MiniLM-L6-v2')
query_result = embedding.embed_query("Machine learning is a branch of artificial intelligence that enables algorithms to uncover hidden patterns within datasets")
query_result

d:\machine-learning\huggingface-langchain\langchain-with-krish\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[-0.06318771839141846,
 -0.0004326881607994437,
 0.052200328558683395,
 0.02694559469819069,
 0.0580267608165741,
 -0.03174372762441635,
 -0.008535039611160755,
 -0.10361742228269577,
 -0.061313193291425705,
 -0.01985531486570835,
 -0.059542927891016006,
 -0.0038338799495249987,
 0.03409747779369354,
 -0.06961182504892349,
 -0.017481308430433273,
 0.0251786932349205,
 -0.03201965242624283,
 0.022269653156399727,
 -0.10581164062023163,
 -0.11773497611284256,
 0.07574648410081863,
 0.004681972786784172,
 -0.0955149382352829,
 0.012706208974123001,
 -0.004206472542136908,
 0.031441256403923035,
 0.06253895908594131,
 0.020741159096360207,
 -0.004080865066498518,
 0.004871193785220385,
 0.058971114456653595,
 0.006204872392117977,
 0.028157679364085197,
 0.031010827049613,
 -0.015287544578313828,
 0.012205218896269798,
 -0.022956999018788338,
 0.07628627121448517,
 0.03015408292412758,
 0.011671667918562889,
 -0.0012005164753645658,
 -0.019160546362400055,
 0.03246565908193588,
 -0.0111287

In [5]:
vector_store = FAISS.from_documents(documents, embedding=embedding)
retriever = vector_store.as_retriever()
retriever.invoke("what is machine learning?")

[Document(id='9c9eb730-8347-4ad1-8779-b6d7a157d166', metadata={'source': 'https://www.geeksforgeeks.org/machine-learning/ml-machine-learning/', 'title': 'What is Machine Learning? - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content='6 min read\nWhat is Machine Learning?Machine learning is a branch of artificial intelligence that enables algorithms to uncover hidden patterns within datasets. It allows them to predict new, similar data without explicit programming for each task. Machine learning finds applications in diverse fields such as image and speech recogniti'),
 Document(id='1ee95fab-67c5-43b9-928f-d02d38774994', metadata={'source': 'https://www.geeksforgeeks.org/machine-learning/ml-machine-learning/', 'title': '

In [6]:
vector_store.similarity_search_with_score("what is machine learning?")

[(Document(id='9c9eb730-8347-4ad1-8779-b6d7a157d166', metadata={'source': 'https://www.geeksforgeeks.org/machine-learning/ml-machine-learning/', 'title': 'What is Machine Learning? - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content='6 min read\nWhat is Machine Learning?Machine learning is a branch of artificial intelligence that enables algorithms to uncover hidden patterns within datasets. It allows them to predict new, similar data without explicit programming for each task. Machine learning finds applications in diverse fields such as image and speech recogniti'),
  np.float32(0.32488203)),
 (Document(id='1ee95fab-67c5-43b9-928f-d02d38774994', metadata={'source': 'https://www.geeksforgeeks.org/machine-learning/ml-m

In [7]:
from langchain.tools.retriever import create_retriever_tool
retriver_tools = create_retriever_tool(retriever, "machine-learning-search", "search any information about machine learning")

In [8]:
retriver_tools.name

'machine-learning-search'

In [9]:
tools = [retriver_tools]
tools

[Tool(name='machine-learning-search', description='search any information about machine learning', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000292E8C3F920>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000292E7AE0090>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x00000292E8C3FC40>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000292E7AE0090>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page

#### Build Groq agent with tools

In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

# Load GROQ API key
groq_api_key = os.getenv('GROQ_API_KEY')

# Import necessary modules 
from langchain_groq import ChatGroq 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor

# Load LLM
llm = ChatGroq(model="llama3-8b-8192", api_key=groq_api_key)

prompt = ChatPromptTemplate.from_messages([
    ('system', "You are an expert ai assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# initialize agent
agent = create_tool_calling_agent( tools=tools, llm=llm, prompt=prompt)

# Executor wrapper
agent_executor = AgentExecutor(agent=agent, tools=tools)
response = agent_executor.invoke({"input": "what is machine learning"})
print(response)



{'input': 'what is machine learning', 'output': 'Here is a direct response based on the provided information:\n\nMachine learning is a branch of artificial intelligence that enables algorithms to uncover hidden patterns within datasets. It allows them to predict new, similar data without explicit programming for each task. Machine learning finds applications in diverse fields such as image and speech recognition, natural language processing, computer vision, and more.'}
